| Model                            | train_loss | val_loss | test_loss | version |
|----------------------------------|------------|----------|-----------|---------|
| Initial                          |     0.1155 |   0.4484 |    4.7467 |     V12 |
| Cross Feature                    |     0.2275 |   0.8123 |    4.4129 |     V13 |
| tsfresh                          |     0.2499 |   0.9606 |    4.4129 |     V14 |
| Attention                        |     0.1141 |   0.4449 |    6.1804 |     V15 |
| CNN+LSTM                         |     0.2341 |   0.8626 |    5.3613 |     V16 |
| CNN                              |     0.1605 |   0.6452 |    4.4129 |     V26 |
| LSTM Fusion                      |     0.2510 |   0.9487 |    6.8452 |     V17 |
| Initial (Bayesian Optimization)  |     0.1255 |   0.5140 |    4.4526 |     V20 |
| Initial (Short-term + Long-term) |     0.2261 |   0.8831 |    4.6863 |     V22 |

In [ ]:
# Read data and aggregate into monthly data

import pandas as pd

train_df = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
# columns: date, date_block_num, shop_id, item_id, item_price, item_cnt_day
train_monthly_df = train_df.groupby(['date_block_num','shop_id','item_id']).agg(sale_cnt_month=('item_cnt_day','sum'),price_month=('item_price','mean')).reset_index()
train_monthly_df['shop_item'] = train_monthly_df['shop_id'].astype('string') + '-' + train_monthly_df['item_id'].astype('string')
shop_item_list = pd.unique(train_monthly_df['shop_item'])
shop_list = pd.unique(train_monthly_df['shop_id'])
item_list = pd.unique(train_monthly_df['item_id'])

print("count of shop_item:",len(shop_item_list))
print('------')
print('count of shop_id:', len(shop_list))
print('min shop_id:', min(shop_list))
print('max shop id:', max(shop_list))
print('------')
print('count item_id:', len(item_list))
print('min item_id:', min(item_list))
print('max item id:', max(item_list))
train_monthly_df

In [ ]:
# get the "sale_cnt sequences" of each "shop_item"
## get (data_seq_dict)

import numpy as np

data_seq_dict = {}
for shop_item in shop_item_list:
    data_seq_dict[shop_item] = [0.0 for i in range(0,34)]
for _, row in train_monthly_df.iterrows():
    data_seq_dict[row['shop_item']][int(row['date_block_num'])] = row['sale_cnt_month']
    
print(len(data_seq_dict))

In [ ]:
# get the "normalized sale_cnt sequences" and ”mean“ of each "shop_item"
## get (data_norm_seq_dict), as well as (data_norm_seq_mean_dict) and (data_norm_seq_stddev_dict)

import statistics

data_norm_seq_dict = {}
data_norm_seq_mean_dict = {}
data_norm_seq_stddev_dict = {}
for k,v in data_seq_dict.items():
    mean = statistics.mean(v)
    data_norm_seq_mean_dict[k] = mean
    stddev = statistics.stdev(v)
    data_norm_seq_stddev_dict[k] = stddev
    if stddev != 0.0:
        data_norm_seq_dict[k] = [(x-mean)/stddev for x in v]
    else:
        data_norm_seq_dict[k] = [0.0 for x in v]

print(len(data_norm_seq_dict))
print(len(data_norm_seq_mean_dict))
print(len(data_norm_seq_stddev_dict))

In [ ]:
# get the "last price" of each "shop_item" as the future price
## shopitem_price_dict

import random

shopitem_price_dict = {}  # {shop_item: [avg_price, current_max_date_block_num, cnt]}
# record price and max date_block_num of each shop_item
for _, row in train_monthly_df.iterrows():
    if row['shop_item'] not in shopitem_price_dict.keys():
        shopitem_price_dict[row['shop_item']] = [row['price_month'], row['date_block_num'], 1]
    else:
        if row['date_block_num'] > shopitem_price_dict[row['shop_item']][1]:
            shopitem_price_dict[row['shop_item']] = [row['price_month'], row['date_block_num'], 1]
        if row['date_block_num'] == shopitem_price_dict[row['shop_item']][1]:
            entry = shopitem_price_dict[row['shop_item']]
            count = entry[2] + 1
            price = (entry[0] * entry[2] + row['price_month']) / count
            shopitem_price_dict[row['shop_item']] = [price, row['date_block_num'], count]
# remove date_block_num and count in the dict to only keep [shop_item, price_month]
for k,v in shopitem_price_dict.items():
    shopitem_price_dict[k] = v[0]

# UT code: printout a random sample to check
print(len(shopitem_price_dict))
idx = random.randint(0,len(shop_item_list))
shop_item = shop_item_list[idx]
print("The last price of shop-item", shop_item, ":", shopitem_price_dict[shop_item])
shop_item = shop_item.split('-')
train_monthly_df[(train_monthly_df['shop_id']==int(shop_item[0])) &
                 (train_monthly_df['item_id']==int(shop_item[1])) &
                 (train_monthly_df['price_month']>0.0)].sort_values(by=['date_block_num'],ascending=[False])

In [ ]:
# get the avg "sale_cnt sequences" of each "item" as default
# In case the shop_item in test dataset does not appear in training dataset, 
# we use the "item avg sale_cnt sequence" as default.
## get (item_seq_dict)

item_seq_dict = {}
item_seq_num_dict = {}
# make sum
for k, v in data_seq_dict.items():
    item = k.split('-')[1]
    if item not in item_seq_dict.keys():
        item_seq_dict[item] = v
        item_seq_num_dict[item] = 1
    else:
        item_seq_dict[item] = [i+j for i, j in zip(item_seq_dict[item], v)]
        item_seq_num_dict[item] += 1
# make average
for item in item_seq_dict.keys():
    item_seq_dict[item] = [element / item_seq_num_dict[item] for element in item_seq_dict[item]]

print(len(item_seq_dict))

In [ ]:
# get the "normalized sale_cnt sequences" and "mean" of each "item"
## get (item_norm_seq_dict), as well as (item_norm_seq_mean_dict) and (item_norm_seq_stddev_dict)

import statistics

item_norm_seq_dict = {}
item_norm_seq_mean_dict = {}
item_norm_seq_stddev_dict = {}
for k,v in item_seq_dict.items():
    mean = statistics.mean(v)
    item_norm_seq_mean_dict[k] = mean
    stddev = statistics.stdev(v)
    item_norm_seq_stddev_dict[k] = stddev
    if stddev != 0:
        item_norm_seq_dict[k] = [(x-mean)/stddev for x in v]
    else:
        item_norm_seq_dict[k] = [0.0 for x in v]

print(len(item_norm_seq_dict))
print(len(item_norm_seq_mean_dict))
print(len(item_norm_seq_stddev_dict))

In [ ]:
# get the "last price" of each "item" as the future price
## get (item_price_dict)

item_price_dict = {}  # {item_id: [avg_price, current_max_date_block_num, cnt]}
# record prices and max date_block_num of each item
for _, row in train_monthly_df.iterrows():
    item_id = str(row['item_id'])
    if item_id not in item_price_dict.keys():
        item_price_dict[item_id] = [row['price_month'], row['date_block_num'], 1]
    else:
        if row['date_block_num'] > item_price_dict[item_id][1]:
            item_price_dict[item_id] = [row['price_month'], row['date_block_num'], 1]
        if row['date_block_num'] == item_price_dict[item_id][1]:
            entry = item_price_dict[item_id]
            count = entry[2] + 1
            price = (entry[0] * entry[2] + row['price_month']) / count
            item_price_dict[item_id] = [price, row['date_block_num'], count]
# remove date_block_num and count in the dict to only keep [item_id, price_month]
for k,v in item_price_dict.items():
    item_price_dict[k] = v[0]
    
# UT code: printout a random sample to check
print(len(item_price_dict))
idx = random.randint(0,len(item_list))
item_id = str(item_list[idx])
print("The last price of item", item_id, ":", item_price_dict[item_id])
tmp_df = train_monthly_df[(train_monthly_df['item_id']==int(item_id))][['date_block_num', 'price_month']]
tmp_df = tmp_df[tmp_df['date_block_num']==tmp_df['date_block_num'].max()][['price_month']].mean()
print(tmp_df)

In [ ]:
# # get the avg "sale_cnt sequences" of each "shop" as default
# # Just for the usage in tsfresh, because the cardinality of shop_item or item_id is too large
# ## get (shop_seq_dict)

# shop_seq_dict = {}
# shop_seq_num_dict = {}
# # make sum
# for k, v in data_seq_dict.items():
#     shop = k.split('-')[0]
#     if shop not in shop_seq_dict.keys():
#         shop_seq_dict[shop] = v
#         shop_seq_num_dict[shop] = 1
#     else:
#         shop_seq_dict[shop] = [i+j for i, j in zip(shop_seq_dict[shop], v)]
#         shop_seq_num_dict[shop] += 1
# # make average
# for shop in shop_seq_dict.keys():
#     shop_seq_dict[shop] = [element / shop_seq_num_dict[shop] for element in shop_seq_dict[shop]]

# print(len(shop_seq_dict))

In [ ]:
# # get the "normalized sale_cnt sequences" and "mean" of each "shop"
# # Just for the usage in tsfresh, because the cardinality of shop_item or item_id is too large
# ## get (shop_norm_seq_dict), as well as (shop_norm_seq_mean_dict) and (shop_norm_seq_stddev_dict)

# import statistics

# shop_norm_seq_dict = {}
# shop_norm_seq_mean_dict = {}
# shop_norm_seq_stddev_dict = {}
# for k,v in shop_seq_dict.items():
#     mean = statistics.mean(v)
#     shop_norm_seq_mean_dict[k] = mean
#     stddev = statistics.stdev(v)
#     shop_norm_seq_stddev_dict[k] = stddev
#     if stddev != 0:
#         shop_norm_seq_dict[k] = [(x-mean)/stddev for x in v]
#     else:
#         shop_norm_seq_dict[k] = [0.0 for x in v]

# print(len(shop_norm_seq_dict))
# print(len(shop_norm_seq_mean_dict))
# print(len(shop_norm_seq_stddev_dict))

In [ ]:
# # get the "last price" of each "shop" as the future price
# # Just for the usage in tsfresh, because the cardinality of shop_item or item_id is too large
# ## get (shop_price_dict)

# shop_price_dict = {}  # {shop_id: [avg_price, current_max_date_block_num, cnt]}
# # record prices and max date_block_num of each shop
# for _, row in train_monthly_df.iterrows():
#     shop_id = str(row['shop_id'])
#     if shop_id not in shop_price_dict.keys():
#         shop_price_dict[shop_id] = [row['price_month'], row['date_block_num'], 1]
#     else:
#         if row['date_block_num'] > shop_price_dict[shop_id][1]:
#             shop_price_dict[shop_id] = [row['price_month'], row['date_block_num'], 1]
#         if row['date_block_num'] == shop_price_dict[shop_id][1]:
#             entry = shop_price_dict[shop_id]
#             count = entry[2] + 1
#             price = (entry[0] * entry[2] + row['price_month']) / count
#             shop_price_dict[shop_id] = [price, row['date_block_num'], count]
# # remove date_block_num and count in the dict to only keep [shop_id, price_month]
# for k,v in shop_price_dict.items():
#     shop_price_dict[k] = v[0]
    
# # UT code: printout a random sample to check
# print(len(shop_price_dict))
# idx = random.randint(0,len(shop_list))
# shop_id = str(shop_list[idx])
# print("The last price of shop", shop_id, ":", shop_price_dict[shop_id])
# tmp_df = train_monthly_df[(train_monthly_df['shop_id']==int(shop_id))][['date_block_num', 'price_month']]
# tmp_df = tmp_df[tmp_df['date_block_num']==tmp_df['date_block_num'].max()][['price_month']].mean()
# print(tmp_df)

In [ ]:
# plot curve

import matplotlib.pyplot as plt

random_idx_array = np.arange(len(shop_item_list))
np.random.shuffle(random_idx_array)
row_random = np.array(list(data_seq_dict.values()))[random_idx_array][0:100,:]
for seq in row_random:
    plt.plot(seq)
plt.xlabel('time step')
plt.ylabel('sale cnt')
plt.show()

In [ ]:
# Data Preprocess

import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K


# parameter
ITEM_VOCAB_LEN = 1000

# Scaler of price
price_scaler = MinMaxScaler(feature_range=(0,1))
price_scaler.fit([[i] for i in train_monthly_df['price_month'].tolist()])

# Split X and Y
X = []
Y = []
sample_weight = []
for k,v in data_norm_seq_dict.items():
    shop_item = k
    static_vec = np.array([int(i) for i in k.split('-')])
    static_vec[1] = static_vec[1] % ITEM_VOCAB_LEN
    price_vec = price_scaler.transform([[shopitem_price_dict[k]]]).reshape((-1,))
    static_vec = np.concatenate([static_vec, price_vec])
    seq_vec = np.array(v).reshape((-1,1))  # (steps, 1)
    X.append([shop_item, static_vec, seq_vec[:-1,:]])
    Y.append(seq_vec[-1,:])
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.3)
X_train_static = np.array([x[1] for x in X_train])
X_train_seq = np.array([x[2] for x in X_train])
Y_train = np.array(Y_train)
X_val_static = np.array([x[1] for x in X_val])
X_val_seq = np.array([x[2] for x in X_val])
Y_val = np.array(Y_val)

# Global Parameters
STEPS = X_train_seq.shape[1]

# generate sample_weight, take mean as sample weight
for entry in X_train:
    shop_item = entry[0]
    sample_weight.append(data_norm_seq_mean_dict[shop_item])
sample_weight = np.array(sample_weight)

print("Training Dataset:")
print("\nX_train_static:")
print(X_train_static.shape)
print(X_train_static)
print("\nX_train_seq:")
print(X_train_seq.shape)
print(X_train_seq)
print("\nsample_weight:")
print(sample_weight.shape)
print(sample_weight)
print("\nY_train:")
print(Y_train.shape)
print(Y_train)
print("\nValidation Dataset:")
print("\nX_val_static:")
print(X_val_static.shape)
print(X_val_static)
print("\nX_val_seq:")
print(X_val_seq.shape)
print(X_val_seq)
print("\nY_val:")
print(Y_val.shape)
print(Y_val)

In [ ]:
# # Initial Model, Wide&Deep

# # Model Parameters
# STEPS = X_train_seq.shape[1]

# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# from math import log
# import tensorflow as tf
# import tensorflow_probability as tfp
# from tensorflow import feature_column
# from tensorflow.nn import relu
# from tensorflow.keras import backend as K
# from tensorflow.keras.models import Sequential
# from tensorflow.keras import Model
# from tensorflow.keras import Input
# from tensorflow.keras.layers import Dense, Lambda, Concatenate, Reshape, Embedding, Conv1D, Add, Flatten, LSTM, Activation, BatchNormalization
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.metrics import mean_squared_error

# # Loss
# def root_mean_squared_error(y_true, y_pred):
#     return K.sqrt(K.mean(K.square(y_pred - y_true))) 

# # Model
# # Wide Component
# input_static_vec = Input(name='input_static', shape = (3,))

# shop_vec = Lambda(lambda x: tf.slice(x, (0,0), (-1,1)))(input_static_vec)
# shop_emb_vec = Embedding(input_dim=60, output_dim=4, input_length=(1,))(shop_vec)
# shop_emb_vec = Flatten()(shop_emb_vec)
# item_vec = Lambda(lambda x: tf.slice(x, (0,1), (-1,1)))(input_static_vec)
# item_emb_vec = Embedding(input_dim=ITEM_VOCAB_LEN, output_dim=4, input_length=(1,))(item_vec)
# item_emb_vec = Flatten()(item_emb_vec)
# price_vec = Lambda(lambda x: tf.slice(x, (0,2), (-1,1)))(input_static_vec)
# wide_vec = Concatenate(axis=-1, name='Concat_wide')([shop_emb_vec, item_emb_vec, price_vec])

# # Deep Component
# input_seq_vec = Input(name='input_seq', shape = (STEPS,1))

# deep_vec = LSTM(units=4, name='LSTM')(input_seq_vec)

# # Fusion Layer
# output_vec = Concatenate(axis=-1, name='Concat_fusion')([wide_vec, deep_vec])
# output_vec = Dense(units=1, activation='relu')(output_vec)

# model = Model(inputs=[input_static_vec,input_seq_vec], outputs=[output_vec])
# model.compile(loss=root_mean_squared_error, optimizer=Adam(learning_rate=0.05))
# model.summary()

# callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001,
#                                               verbose=1, mode='min', patience=3,
#                                               restore_best_weights=True)]

# history = model.fit(x={'input_static': X_train_static, 'input_seq': X_train_seq}, 
#                     y=Y_train,
#                     sample_weight=sample_weight,
#                     validation_data=({'input_static':X_val_static,'input_seq':X_val_seq}, Y_val), 
#                     epochs=100, 
#                     batch_size=1024*1024,
#                     callbacks=callbacks)

# # Original Model, loss: 0.1155 - val_loss: 0.4484 - test_loss: 4.7467 (V12)

In [ ]:
# # Cross Feature Model

# # Model Parameters
# STEPS = X_train_seq.shape[1]

# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# from math import log
# import tensorflow as tf
# import tensorflow_probability as tfp
# from tensorflow import feature_column
# from tensorflow.nn import relu
# from tensorflow.keras import backend as K
# from tensorflow.keras.models import Sequential
# from tensorflow.keras import Model
# from tensorflow.keras import Input
# from tensorflow.keras.layers import Dense, Lambda, Concatenate, Reshape, Embedding, Conv1D, Add, Flatten, LSTM, Activation, BatchNormalization, Multiply
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.metrics import mean_squared_error

# # Loss
# def root_mean_squared_error(y_true, y_pred):
#     return K.sqrt(K.mean(K.square(y_pred - y_true))) 

# # Model
# # Wide Component
# input_static_vec = Input(name='input_static', shape = (3,))

# shop_vec = Lambda(lambda x: tf.slice(x, (0,0), (-1,1)))(input_static_vec)
# shop_emb_vec = Embedding(input_dim=60, output_dim=4, input_length=(1,))(shop_vec)
# shop_emb_vec = Flatten()(shop_emb_vec)
# item_vec = Lambda(lambda x: tf.slice(x, (0,1), (-1,1)))(input_static_vec)
# item_emb_vec = Embedding(input_dim=ITEM_VOCAB_LEN, output_dim=4, input_length=(1,))(item_vec)
# item_emb_vec = Flatten()(item_emb_vec)
# price_vec = Lambda(lambda x: tf.slice(x, (0,2), (-1,1)))(input_static_vec)
# wide_vec = Concatenate(axis=-1, name='Concat_wide')([shop_emb_vec, item_emb_vec, price_vec])

# # Cross Feature
# shop_item_vec = Multiply()([shop_emb_vec, item_emb_vec])
# item_price_vec = Multiply()([item_emb_vec, price_vec])

# # Deep Component
# input_seq_vec = Input(name='input_seq', shape = (STEPS,1))

# deep_vec = LSTM(units=4, name='LSTM')(input_seq_vec)

# # Fusion Layer
# output_vec = Concatenate(axis=-1, name='Concat_fusion')([wide_vec, shop_item_vec, item_price_vec, deep_vec])
# output_vec = Dense(units=1, activation='relu')(output_vec)

# model = Model(inputs=[input_static_vec,input_seq_vec], outputs=[output_vec])
# model.compile(loss=root_mean_squared_error, optimizer=Adam(learning_rate=0.05))
# model.summary()

# callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001,
#                                               verbose=1, mode='min', patience=3,
#                                               restore_best_weights=True)]

# history = model.fit(x={'input_static': X_train_static, 'input_seq': X_train_seq}, 
#                     y=Y_train,
#                     sample_weight=sample_weight,
#                     validation_data=({'input_static':X_val_static,'input_seq':X_val_seq}, Y_val), 
#                     epochs=100, 
#                     batch_size=1024*1024,
#                     callbacks=callbacks)

# # Cross Feature Model - loss: 0.2275 - val_loss: 0.8123 - test_loss: 4.41291 (V13)

In [ ]:
# # tsfresh Model

# # generate the tsfresh feature of shop (shop_item cardinality too large)
# # Ref: https://tsfresh.readthedocs.io/en/latest/


# ### DATA PREPROCESS ###

# from tsfresh import extract_relevant_features

# # preprocess shop dataframe (take shop_id as example because cardinality of shop_item or item_id is too large)
# X_shop = []
# Y_shop = []
# for k,v in shop_norm_seq_dict.items():
#     shop_id = k
#     static_vec = np.array([int(k)])
#     price_vec = price_scaler.transform([[shop_price_dict[k]]]).reshape((-1,))
#     static_vec = np.concatenate([static_vec, price_vec])
#     seq_vec = np.array(v).reshape((-1,1))  # (steps, 1)
#     X_shop.append([shop_id, static_vec, seq_vec[:-1,:]])
#     Y_shop.append(seq_vec[-1,:])

    
# # generate tsfresh x and y
# tsfresh_x_df = pd.DataFrame(columns=['shop_id', 'date_block_num', 'sale_cnt_month'])
# tsfresh_y_df = pd.DataFrame(columns=['shop_id', 'sale_cnt_month'])
# for idx in range(0,len(X_shop)):
#     shop_id = X_shop[idx][0]
#     seq_array = np.squeeze(X_shop[idx][2])  # (33,)
#     for step in range(0,len(seq_array)):
#         tsfresh_x_df = tsfresh_x_df.append({'shop_id': shop_id, 'date_block_num': step, 'sale_cnt_month':seq_array[step]}, ignore_index=True)
#     tsfresh_y_df = tsfresh_y_df.append({'shop_id': shop_id, 'sale_cnt_month':Y_shop[idx][0]}, ignore_index=True)
# tsfresh_y_df = tsfresh_y_df.set_index('shop_id')['sale_cnt_month']


# # generate tsfresh features
# features_filtered = extract_relevant_features(tsfresh_x_df, tsfresh_y_df,
#                                               column_id='shop_id', column_sort='date_block_num')
# print("The shape of tsfresh dataframe:")
# print(features_filtered.shape)  # (shop_num, feature_num)


# # concat the tsfresh back to X_train
# ## X_train_static
# tsfresh_feature_list = []
# for entry in X_train_static:
#     shop_id = int(entry[0])
#     tsfresh_feature = features_filtered.loc[[str(shop_id)]].to_numpy()  # (1, tsfresh_num)
#     tsfresh_feature_list.append(tsfresh_feature)
# tsfresh_feature_array = np.concatenate(tsfresh_feature_list, axis=0)
# print("The shape of X tsfresh array shape:")
# print(tsfresh_feature_array.shape)  # (N, tsfresh_num)

# X_train_static = np.concatenate([X_train_static, tsfresh_feature_array], axis=1)
# print("The shape of X_train_static:")
# print(X_train_static.shape)  # (N, static_dim+tsfresh_num)

# ## X_val_static
# tsfresh_feature_list = []
# for entry in X_val_static:
#     shop_id = int(entry[0])
#     tsfresh_feature = features_filtered.loc[[str(shop_id)]].to_numpy()  # (1, feature_num)
#     tsfresh_feature_list.append(tsfresh_feature)
# tsfresh_feature_array = np.concatenate(tsfresh_feature_list, axis=0)
# print("The shape of X tsfresh array shape:")
# print(tsfresh_feature_array.shape)  # (N, feature_num)

# X_val_static = np.concatenate([X_val_static, tsfresh_feature_array], axis=1)
# print("The shape of X_val_static:")
# print(X_val_static.shape)  # (N, static_dim+tsfresh_num)


# ### MODEL ###

# # Model Parameters
# STEPS = X_train_seq.shape[1]
# STATIC_LEN = X_train_static.shape[1]

# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# from math import log
# import tensorflow as tf
# import tensorflow_probability as tfp
# from tensorflow import feature_column
# from tensorflow.nn import relu
# from tensorflow.keras import backend as K
# from tensorflow.keras.models import Sequential
# from tensorflow.keras import Model
# from tensorflow.keras import Input
# from tensorflow.keras.layers import Dense, Lambda, Concatenate, Reshape, Embedding, Conv1D, Add, Flatten, LSTM, Activation, BatchNormalization
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.metrics import mean_squared_error

# # Loss
# def root_mean_squared_error(y_true, y_pred):
#     return K.sqrt(K.mean(K.square(y_pred - y_true))) 

# # Model
# # Wide Component
# input_static_vec = Input(name='input_static', shape = (STATIC_LEN,))

# shop_vec = Lambda(lambda x: tf.slice(x, (0,0), (-1,1)))(input_static_vec)
# shop_emb_vec = Embedding(input_dim=60, output_dim=4, input_length=(1,))(shop_vec)
# shop_emb_vec = Flatten()(shop_emb_vec)
# item_vec = Lambda(lambda x: tf.slice(x, (0,1), (-1,1)))(input_static_vec)
# item_emb_vec = Embedding(input_dim=ITEM_VOCAB_LEN, output_dim=4, input_length=(1,))(item_vec)
# item_emb_vec = Flatten()(item_emb_vec)
# num_static_vec = Lambda(lambda x: tf.slice(x, (0,2), (-1,STATIC_LEN-2)))(input_static_vec)
# wide_vec = Concatenate(axis=-1, name='Concat_wide')([shop_emb_vec, item_emb_vec, num_static_vec])

# # Deep Component
# input_seq_vec = Input(name='input_seq', shape = (STEPS,1))

# deep_vec = LSTM(units=4, name='LSTM')(input_seq_vec)

# # Fusion Layer
# output_vec = Concatenate(axis=-1, name='Concat_fusion')([wide_vec, deep_vec])
# output_vec = Dense(units=1, activation='relu')(output_vec)

# model = Model(inputs=[input_static_vec,input_seq_vec], outputs=[output_vec])
# model.compile(loss=root_mean_squared_error, optimizer=Adam(learning_rate=0.05))
# model.summary()

# callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001,
#                                               verbose=1, mode='min', patience=3,
#                                               restore_best_weights=True)]

# history = model.fit(x={'input_static': X_train_static, 'input_seq': X_train_seq}, 
#                     y=Y_train,
#                     sample_weight=sample_weight,
#                     validation_data=({'input_static':X_val_static,'input_seq':X_val_seq}, Y_val), 
#                     epochs=100, 
#                     batch_size=1024*1024,
#                     callbacks=callbacks)

# # tsfresh Model, loss: 0.2499 - val_loss: 0.9606 - test_loss: 4.4129 (V14)

In [ ]:
# # Attention Model

# # Model Parameters
# STEPS = X_train_seq.shape[1]

# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# from math import log
# import tensorflow as tf
# import tensorflow_probability as tfp
# from tensorflow import feature_column
# from tensorflow.nn import relu
# from tensorflow.keras import backend as K
# from tensorflow.keras.models import Sequential
# from tensorflow.keras import Model
# from tensorflow.keras import Input
# from tensorflow.keras.layers import Dense, Lambda, Concatenate, Reshape, Embedding, Conv1D, Add, Flatten, LSTM, Activation, BatchNormalization, Attention
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.metrics import mean_squared_error

# # Loss
# def root_mean_squared_error(y_true, y_pred):
#     return K.sqrt(K.mean(K.square(y_pred - y_true))) 

# # Model
# # Wide Component
# input_static_vec = Input(name='input_static', shape = (3,))

# shop_vec = Lambda(lambda x: tf.slice(x, (0,0), (-1,1)))(input_static_vec)
# shop_emb_vec = Embedding(input_dim=60, output_dim=4, input_length=(1,))(shop_vec)
# shop_emb_vec = Flatten()(shop_emb_vec)
# item_vec = Lambda(lambda x: tf.slice(x, (0,1), (-1,1)))(input_static_vec)
# item_emb_vec = Embedding(input_dim=ITEM_VOCAB_LEN, output_dim=4, input_length=(1,))(item_vec)
# item_emb_vec = Flatten()(item_emb_vec)
# price_vec = Lambda(lambda x: tf.slice(x, (0,2), (-1,1)))(input_static_vec)
# wide_vec = Concatenate(axis=-1, name='Concat_wide')([shop_emb_vec, item_emb_vec, price_vec])

# # Deep Component
# input_seq_vec = Input(name='input_seq', shape = (STEPS,1))  # (N, steps, 1)
# # self-attention
# deep_vec = Attention()([input_seq_vec, input_seq_vec, input_seq_vec], return_attention_scores=False)  # (N, steps, 1)
# deep_vec = LSTM(units=4, name='LSTM')(deep_vec)

# # Fusion Layer
# output_vec = Concatenate(axis=-1, name='Concat_fusion')([wide_vec, deep_vec])
# output_vec = Dense(units=1, activation='relu')(output_vec)

# model = Model(inputs=[input_static_vec,input_seq_vec], outputs=[output_vec])
# model.compile(loss=root_mean_squared_error, optimizer=Adam(learning_rate=0.05))
# model.summary()

# callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001,
#                                               verbose=1, mode='min', patience=3,
#                                               restore_best_weights=True)]

# history = model.fit(x={'input_static': X_train_static, 'input_seq': X_train_seq}, 
#                     y=Y_train,
#                     sample_weight=sample_weight,
#                     validation_data=({'input_static':X_val_static,'input_seq':X_val_seq}, Y_val), 
#                     epochs=100, 
#                     batch_size=1024*1024,
#                     callbacks=callbacks)

# # Attention Model, loss: 0.1141 - val_loss: 0.4449 - test_loss: 6.1804 (V15)

In [ ]:
# # CNN+LSTM Model

# # Model Parameters
# STEPS = X_train_seq.shape[1]

# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# from math import log
# import tensorflow as tf
# import tensorflow_probability as tfp
# from tensorflow import feature_column
# from tensorflow.nn import relu
# from tensorflow.keras import backend as K
# from tensorflow.keras.models import Sequential
# from tensorflow.keras import Model
# from tensorflow.keras import Input
# from tensorflow.keras.layers import Dense, Lambda, Concatenate, Reshape, Embedding, Conv1D, Add, Flatten, LSTM, Activation, BatchNormalization
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.metrics import mean_squared_error

# # Loss
# def root_mean_squared_error(y_true, y_pred):
#     return K.sqrt(K.mean(K.square(y_pred - y_true))) 

# # Model
# # Wide Component
# input_static_vec = Input(name='input_static', shape = (3,))

# shop_vec = Lambda(lambda x: tf.slice(x, (0,0), (-1,1)))(input_static_vec)
# shop_emb_vec = Embedding(input_dim=60, output_dim=4, input_length=(1,))(shop_vec)
# shop_emb_vec = Flatten()(shop_emb_vec)
# item_vec = Lambda(lambda x: tf.slice(x, (0,1), (-1,1)))(input_static_vec)
# item_emb_vec = Embedding(input_dim=ITEM_VOCAB_LEN, output_dim=4, input_length=(1,))(item_vec)
# item_emb_vec = Flatten()(item_emb_vec)
# price_vec = Lambda(lambda x: tf.slice(x, (0,2), (-1,1)))(input_static_vec)
# wide_vec = Concatenate(axis=-1, name='Concat_wide')([shop_emb_vec, item_emb_vec, price_vec])

# # Deep Component
# input_seq_vec = Input(name='input_seq', shape = (STEPS,1))

# deep_vec = Conv1D(filters=4, kernel_size=3, strides=1, padding='valid', name='CNN')(input_seq_vec)  # (N,31,4)
# deep_vec = LSTM(units=4, name='LSTM')(deep_vec)  # (N,4)

# # Fusion Layer
# output_vec = Concatenate(axis=-1, name='Concat_fusion')([wide_vec, deep_vec])
# output_vec = Dense(units=1, activation='relu')(output_vec)

# model = Model(inputs=[input_static_vec,input_seq_vec], outputs=[output_vec])
# model.compile(loss=root_mean_squared_error, optimizer=Adam(learning_rate=0.05))
# model.summary()

# callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001,
#                                               verbose=1, mode='min', patience=3,
#                                               restore_best_weights=True)]

# history = model.fit(x={'input_static': X_train_static, 'input_seq': X_train_seq}, 
#                     y=Y_train,
#                     sample_weight=sample_weight,
#                     validation_data=({'input_static':X_val_static,'input_seq':X_val_seq}, Y_val), 
#                     epochs=100, 
#                     batch_size=1024*1024,
#                     callbacks=callbacks)

# # CNN+LSTM Model, - loss: 0.2341 - val_loss: 0.8626 - test_loss: 5.3613 (V16)

In [ ]:
# CNN Model

# Model Parameters
STEPS = X_train_seq.shape[1]

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from math import log
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow import feature_column
from tensorflow.nn import relu
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Lambda, Concatenate, Reshape, Embedding, Conv1D, Add, Flatten, LSTM, Activation, BatchNormalization, AveragePooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import mean_squared_error

# Loss
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 

# Model
# Wide Component
input_static_vec = Input(name='input_static', shape = (3,))

shop_vec = Lambda(lambda x: tf.slice(x, (0,0), (-1,1)))(input_static_vec)
shop_emb_vec = Embedding(input_dim=60, output_dim=4, input_length=(1,))(shop_vec)
shop_emb_vec = Flatten()(shop_emb_vec)
item_vec = Lambda(lambda x: tf.slice(x, (0,1), (-1,1)))(input_static_vec)
item_emb_vec = Embedding(input_dim=ITEM_VOCAB_LEN, output_dim=4, input_length=(1,))(item_vec)
item_emb_vec = Flatten()(item_emb_vec)
price_vec = Lambda(lambda x: tf.slice(x, (0,2), (-1,1)))(input_static_vec)
wide_vec = Concatenate(axis=-1, name='Concat_wide')([shop_emb_vec, item_emb_vec, price_vec])

# Deep Component
input_seq_vec = Input(name='input_seq', shape = (STEPS,1))

deep_vec = Conv1D(filters=4, kernel_size=3, strides=1, padding='valid', name='CNN_1')(input_seq_vec)  # (N,31,4)
deep_vec = AveragePooling1D(pool_size=5, strides=5, padding='valid', name='Pooling_1')(deep_vec)  # (N,6,4)
deep_vec = Conv1D(filters=4, kernel_size=3, strides=1, padding='valid', name='CNN_2')(deep_vec)  # (N,4,4)
deep_vec = AveragePooling1D(pool_size=4, strides=4, padding='valid', name='Pooling_2')(deep_vec)  # (N,1,4)
deep_vec = Flatten()(deep_vec)  # (N,4)


# Fusion Layer
output_vec = Concatenate(axis=-1, name='Concat_fusion')([wide_vec, deep_vec])
output_vec = Dense(units=1, activation='relu')(output_vec)

model = Model(inputs=[input_static_vec,input_seq_vec], outputs=[output_vec])
model.compile(loss=root_mean_squared_error, optimizer=Adam(learning_rate=0.05))
model.summary()

callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001,
                                              verbose=1, mode='min', patience=3,
                                              restore_best_weights=True)]

history = model.fit(x={'input_static': X_train_static, 'input_seq': X_train_seq}, 
                    y=Y_train,
                    sample_weight=sample_weight,
                    validation_data=({'input_static':X_val_static,'input_seq':X_val_seq}, Y_val), 
                    epochs=100, 
                    batch_size=1024*1024,
                    callbacks=callbacks)

# CNN Model, - loss: 0.1605 - val_loss: 0.6452 - test_loss: 4.4129 (V26)

In [ ]:
# # LSTM Fusion Model

# # Model Parameters
# STEPS = X_train_seq.shape[1]

# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# from math import log
# import tensorflow as tf
# import tensorflow_probability as tfp
# from tensorflow import feature_column
# from tensorflow.nn import relu
# from tensorflow.keras import backend as K
# from tensorflow.keras.models import Sequential
# from tensorflow.keras import Model
# from tensorflow.keras import Input
# from tensorflow.keras.layers import Dense, Lambda, Concatenate, Reshape, Embedding, Conv1D, Add, Flatten, LSTM, Activation, BatchNormalization, RepeatVector
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.metrics import mean_squared_error

# # Loss
# def root_mean_squared_error(y_true, y_pred):
#     return K.sqrt(K.mean(K.square(y_pred - y_true))) 

# # Model
# # Wide Component
# input_static_vec = Input(name='input_static', shape = (3,))

# shop_vec = Lambda(lambda x: tf.slice(x, (0,0), (-1,1)))(input_static_vec)
# shop_emb_vec = Embedding(input_dim=60, output_dim=4, input_length=(1,))(shop_vec)
# shop_emb_vec = Flatten()(shop_emb_vec)
# item_vec = Lambda(lambda x: tf.slice(x, (0,1), (-1,1)))(input_static_vec)
# item_emb_vec = Embedding(input_dim=ITEM_VOCAB_LEN, output_dim=4, input_length=(1,))(item_vec)
# item_emb_vec = Flatten()(item_emb_vec)
# price_vec = Lambda(lambda x: tf.slice(x, (0,2), (-1,1)))(input_static_vec)
# wide_vec = Concatenate(axis=-1, name='Concat_wide')([shop_emb_vec, item_emb_vec, price_vec])  # (N, dim)

# # Deep Component
# input_seq_vec = Input(name='input_seq', shape = (STEPS,1))  # (N, steps, 1)

# # Fusion Layer by LSTM
# ## prepare Fusion input
# wide_vec = RepeatVector(STEPS)(wide_vec)  # (N, steps, dim)
# fusion_vec = Concatenate(axis=-1, name='Concat_fusion')([input_seq_vec, wide_vec])  # (N, steps, 1+dim)
# ## fusion layer
# output_vec = LSTM(units=4, name='LSTM_fusion')(fusion_vec)  # (N,4)
# output_vec = Dense(units=1, activation='relu')(output_vec)

# model = Model(inputs=[input_static_vec,input_seq_vec], outputs=[output_vec])
# model.compile(loss=root_mean_squared_error, optimizer=Adam(learning_rate=0.05))
# model.summary()

# callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001,
#                                               verbose=1, mode='min', patience=3,
#                                               restore_best_weights=True)]

# history = model.fit(x={'input_static': X_train_static, 'input_seq': X_train_seq}, 
#                     y=Y_train,
#                     sample_weight=sample_weight,
#                     validation_data=({'input_static':X_val_static,'input_seq':X_val_seq}, Y_val), 
#                     epochs=100, 
#                     batch_size=1024*1024,
#                     callbacks=callbacks)

# # LSTM Fusion Model, - loss: 0.2510 - val_loss: 0.9487 - test_loss: 6.8452 (V17)

In [ ]:
# # Initial Model - Bayesian Optimization

# # Model Parameters
# STEPS = X_train_seq.shape[1]

# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# from math import log
# import tensorflow as tf
# import tensorflow_probability as tfp
# from tensorflow import feature_column
# from tensorflow.nn import relu
# from tensorflow.keras import backend as K
# from tensorflow.keras.models import Sequential
# from tensorflow.keras import Model
# from tensorflow.keras import Input
# from tensorflow.keras.layers import Dense, Lambda, Concatenate, Reshape, Embedding, Conv1D, Add, Flatten, LSTM, Activation, BatchNormalization
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.metrics import mean_squared_error
# import keras_tuner as kt

# def model_builder(hp):
#     # Optimized Hyperparameter
#     hp_emb_size = hp.Choice('emb_size', values=[2,4,8,16])
#     hp_lstm_units = hp.Choice('lstm_units', values=[1,2,4,8,16])
#     hp_learning_rate = hp.Float('learning_rate', min_value=0.002, max_value=0.1, step=0.002)
    
#     # Loss
#     def root_mean_squared_error(y_true, y_pred):
#         return K.sqrt(K.mean(K.square(y_pred - y_true))) 

#     # Model
#     # Wide Component
#     input_static_vec = Input(name='input_static', shape = (3,))

#     shop_vec = Lambda(lambda x: tf.slice(x, (0,0), (-1,1)))(input_static_vec)
#     shop_emb_vec = Embedding(input_dim=60, output_dim=hp_emb_size, input_length=(1,))(shop_vec)
#     shop_emb_vec = Flatten()(shop_emb_vec)
#     item_vec = Lambda(lambda x: tf.slice(x, (0,1), (-1,1)))(input_static_vec)
#     item_emb_vec = Embedding(input_dim=ITEM_VOCAB_LEN, output_dim=hp_emb_size, input_length=(1,))(item_vec)
#     item_emb_vec = Flatten()(item_emb_vec)
#     price_vec = Lambda(lambda x: tf.slice(x, (0,2), (-1,1)))(input_static_vec)
#     wide_vec = Concatenate(axis=-1, name='Concat_wide')([shop_emb_vec, item_emb_vec, price_vec])

#     # Deep Component
#     input_seq_vec = Input(name='input_seq', shape = (STEPS,1))
#     deep_vec = LSTM(units=hp_lstm_units, name='LSTM')(input_seq_vec)

#     # Fusion Layer
#     output_vec = Concatenate(axis=-1, name='Concat_fusion')([wide_vec, deep_vec])
#     output_vec = Dense(units=1, activation='relu')(output_vec)

#     model = Model(inputs=[input_static_vec,input_seq_vec], outputs=[output_vec])
#     model.compile(loss=root_mean_squared_error, optimizer=Adam(learning_rate=hp_learning_rate))
    
#     return model

# # Tuner
# tuner = kt.BayesianOptimization(hypermodel=model_builder,
#                                 objective='val_loss',
#                                 max_trials=10,
#                                 num_initial_points=3,
#                                 project_name='Initial_Model_Tuner')

# callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001,
#                                               verbose=1, mode='min', patience=3,
#                                               restore_best_weights=True)]

# tuner.search(x={'input_static': X_train_static, 'input_seq': X_train_seq}, 
#              y=Y_train,
#              sample_weight=sample_weight,
#              validation_data=({'input_static':X_val_static,'input_seq':X_val_seq}, Y_val), 
#              epochs=100, 
#              batch_size=1024*1024,
#              callbacks=callbacks)
# print('\n')
# tuner.search_space_summary()
# print('\n')
# tuner.results_summary()

# # Get the optimal hyperparameters
# best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
# print(type(best_hps))
# print(best_hps.values)
# print(f"""
# The best emb_size is {best_hps.get('emb_size')} 
# The best lstm_units is {best_hps.get('lstm_units')}
# The best learning_rate is {best_hps.get('learning_rate')}
# """)

# # Get the optimal model
# model = tuner.hypermodel.build(best_hps)  # better than use get_best_model, which retrieves the best model in all the trails, not the best hyperparameter after the last trail
# model.summary()
# history = model.fit(x={'input_static': X_train_static, 'input_seq': X_train_seq}, 
#                     y=Y_train,
#                     sample_weight=sample_weight,
#                     validation_data=({'input_static':X_val_static,'input_seq':X_val_seq}, Y_val), 
#                     epochs=100, 
#                     batch_size=1024*1024,
#                     callbacks=callbacks)

# # Initial Model (Bayesian Optimization), - loss: 0.1255 - val_loss: 0.5140 - test_loss: 4.4526 (V20)

In [ ]:
# # Initial Model, Short-term + Long-term

# # Model Parameters
# STEPS = X_train_seq.shape[1]

# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# from math import log
# import tensorflow as tf
# import tensorflow_probability as tfp
# from tensorflow import feature_column
# from tensorflow.nn import relu
# from tensorflow.keras import backend as K
# from tensorflow.keras.models import Sequential
# from tensorflow.keras import Model
# from tensorflow.keras import Input
# from tensorflow.keras.layers import Dense, Lambda, Concatenate, Reshape, Embedding, Conv1D, Add, Flatten, LSTM, Activation, BatchNormalization
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.metrics import mean_squared_error

# # Loss
# def root_mean_squared_error(y_true, y_pred):
#     return K.sqrt(K.mean(K.square(y_pred - y_true))) 

# # Model
# # Wide Component
# input_static_vec = Input(name='input_static', shape = (3,))

# shop_vec = Lambda(lambda x: tf.slice(x, (0,0), (-1,1)))(input_static_vec)
# shop_emb_vec = Embedding(input_dim=60, output_dim=4, input_length=(1,))(shop_vec)
# shop_emb_vec = Flatten()(shop_emb_vec)
# item_vec = Lambda(lambda x: tf.slice(x, (0,1), (-1,1)))(input_static_vec)
# item_emb_vec = Embedding(input_dim=ITEM_VOCAB_LEN, output_dim=4, input_length=(1,))(item_vec)
# item_emb_vec = Flatten()(item_emb_vec)
# price_vec = Lambda(lambda x: tf.slice(x, (0,2), (-1,1)))(input_static_vec)
# wide_vec = Concatenate(axis=-1, name='Concat_wide')([shop_emb_vec, item_emb_vec, price_vec])

# # Deep Component - Long-Term
# input_seq_vec = Input(name='input_seq', shape = (STEPS,1))
# deep_long_vec = LSTM(units=4, name='LSTM_long')(input_seq_vec)

# # Deep Component - Short-Term
# deep_short_vec = Lambda(lambda x: tf.slice(x,(0,STEPS-3,0),(-1,3,-1)), name='short_term_Slice')(input_seq_vec)
# deep_short_vec = LSTM(units=4, name='LSTM_short')(deep_short_vec)

# # Fusion Layer
# output_vec = Concatenate(axis=-1, name='Concat_fusion')([wide_vec, deep_short_vec, deep_long_vec])
# output_vec = Dense(units=1, activation='relu')(output_vec)

# model = Model(inputs=[input_static_vec,input_seq_vec], outputs=[output_vec])
# model.compile(loss=root_mean_squared_error, optimizer=Adam(learning_rate=0.05))
# model.summary()

# callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001,
#                                               verbose=1, mode='min', patience=3,
#                                               restore_best_weights=True)]

# history = model.fit(x={'input_static': X_train_static, 'input_seq': X_train_seq}, 
#                     y=Y_train,
#                     sample_weight=sample_weight,
#                     validation_data=({'input_static':X_val_static,'input_seq':X_val_seq}, Y_val), 
#                     epochs=100, 
#                     batch_size=1024*1024,
#                     callbacks=callbacks)

# # Initial Model (Short-term + Long-term), loss: 0.2261 - val_loss: 0.8831 - test_loss: 4.6863 (V22)

In [ ]:
tf.keras.utils.plot_model(model, to_file='model_graph.png', show_shapes=True, show_layer_names=True)

In [ ]:
plt.plot(history.history['loss'])
plt.show()

In [ ]:
plt.plot(history.history['val_loss'])
plt.show()

In [ ]:
# Prepare Test dataset

test_df = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv').set_index('ID')
test_df['shop_item'] = test_df['shop_id'].astype('string') + '-' + test_df['item_id'].astype('string')

X_test_static = []
X_test_seq = []
for _, row in test_df.iterrows():
    shop_item = row['shop_item']
    shop_id = str(row['shop_id'])
    item_id = str(row['item_id'])
    
    # get sale_cnt_seq
    if shop_item in data_norm_seq_dict.keys():
        sale_norm_cnt_seq = data_norm_seq_dict[shop_item][1:]
    else:
        if item_id in item_norm_seq_dict.keys():
            sale_norm_cnt_seq = item_norm_seq_dict[item_id][1:]
        else:
            sale_norm_cnt_seq = [0.0 for i in range(0,STEPS)]
    
    # get price
    if shop_item in shopitem_price_dict.keys():
        price = shopitem_price_dict[shop_item]
    else:
        if item_id in item_price_dict.keys():
            price = item_price_dict[item_id]
        else:
            price = 0.0
    
    # generate the sample
    static_vec = np.array([int(shop_id), int(item_id) % ITEM_VOCAB_LEN])
    price_vec = price_scaler.transform([[price]]).reshape((-1,))
    static_vec = np.concatenate([static_vec, price_vec])
    seq_vec = np.array(sale_norm_cnt_seq).reshape((-1,1))  # (steps, 1)
    X_test_static.append(static_vec)
    X_test_seq.append(seq_vec)
X_test_static = np.array(X_test_static)
X_test_seq = np.array(X_test_seq)

# ## [tsfresh Model] ##
# # concat tsfresh feature to X_test_static
# tsfresh_feature_list = []
# for entry in X_test_static:
#     shop_id = int(entry[0])
#     tsfresh_feature = features_filtered.loc[[str(shop_id)]].to_numpy()  # (1, tsfresh_num)
#     tsfresh_feature_list.append(tsfresh_feature)
# tsfresh_feature_array = np.concatenate(tsfresh_feature_list, axis=0)
# print("The shape of X_test tsfresh array shape:")
# print(tsfresh_feature_array.shape)  # (N, tsfresh_num)

# X_test_static = np.concatenate([X_test_static, tsfresh_feature_array], axis=1)
# ## [tsfresh Model] ##


print('X_test_static:')
print(X_test_static.shape)
print(X_test_static)
print('\nX_test_seq:')
print(X_test_seq.shape)
print(X_test_seq)

In [ ]:
pred = model.predict({'input_static':X_test_static, 'input_seq':X_test_seq})
print(pred.shape)
print(pred)

In [ ]:
# recover the prediction (de-normalized)
results = []
for i in range(0,pred.shape[0]):
    shop_id = str(test_df.iloc[i]['shop_id'])
    item_id = str(test_df.iloc[i]['item_id'])
    shop_item = shop_id + '-' + item_id
    if shop_item in data_norm_seq_dict.keys():
        mean = data_norm_seq_mean_dict[shop_item]
        stddev = data_norm_seq_stddev_dict[shop_item]
        result = pred[i][0] * stddev + mean
    else:
        if item_id in item_norm_seq_dict.keys():
            mean = item_norm_seq_mean_dict[item_id]
            stddev = item_norm_seq_stddev_dict[item_id]
            result = pred[i][0] * stddev + mean
        else:
            result = 0.0
    if result < 0.0:
        result = 0.0
    results.append(result)
results[0:10]

In [ ]:
submission_df = pd.DataFrame(results, columns=['item_cnt_month'])
submission_df.index.name = 'ID'
submission_df

In [ ]:
submission_df.to_csv('/kaggle/working/submission.csv')

In [ ]:
# # Investigate the importance of "yesterday Wednesday" and "last Thurseday"
# data_seq_array = np.array(list(data_seq_dict.values()))  # (N, 34)
# Conclusion: RMSE_last_day = 3.47, RMSE_last_Sunday = 3.83, 
# the "recent time" is more important, however it is not significant.

# # rmse of last step
# rmse_last_step = np.sqrt(np.mean((data_seq_array[:,-7:] - data_seq_array[:,-8:-1])**2))
# print(rmse_last_step)

# # rmse of the corresponding step in last period
# rmse_crspd_last_perid = np.sqrt(np.mean((data_seq_array[:,-7:] - data_seq_array[:,-14:-7])**2))
# print(rmse_crspd_last_perid)